# 0207 Course Schedule

## Problem

There are a total of `numCourses` courses you have to take, labeled from `0` to `numCourses - 1`. You are given an array prerequisites where `prerequisites[i] = [ai, bi]` indicates that you must take course `bi` first if you want to take course `ai`.

For example, the pair `[0, 1]`, indicates that to take course `0` you have to first take course `1`.
Return `true` if you can finish all courses. Otherwise, return `false`.

### Examples 

Example 1:

``` text
Input: numCourses = 2, prerequisites = [[1,0]]
Output: true
Explanation: There are a total of 2 courses to take. 
To take course 1 you should have finished course 0. So it is possible.
```

Example 2:

```text
Input: numCourses = 2, prerequisites = [[1,0],[0,1]]
Output: false
Explanation: There are a total of 2 courses to take. 
To take course 1 you should have finished course 0, and to take course 0 you should also have finished course 1. So it is impossible.
```

### Constraints:

```text
1 <= numCourses <= 2000
0 <= prerequisites.length <= 5000
prerequisites[i].length == 2
0 <= ai, bi < numCourses
All the pairs prerequisites[i] are unique.

```
 
### Follow-up


## Analysis
- when is it not possible to finish?
  - when there are loops in the dependency graph
- how to detect if such a dependency graph has loops or not?
  - traverse the graph using DFS or BFS
  - mark visited nodes
  - if already visited, then there is a loop in the directed graph

## BFS Solution

In [41]:
def canFinish(numCourses, prerequisites):

    # build graph from given number of courses and prerequisites
    # here we represent graph as a dictionary
    def buildGraph(numCourses, prerequisites):
        graph = {}
        for i in range(numCourses):
            graph[i] = []
        for i in range(len(prerequisites)):
            graph[prerequisites[i][0]].append(prerequisites[i][1])
        return graph 
    
    # check if there is a cycle in the graph
    def hasCycle(graph):
        
        visited = set()
        visited.add(0)
        queue = [0]

        while queue:
            node = queue.pop(0)

            # explore all the neighbors of the node
            for neighbor in graph[node]:
                if neighbor in visited:
                    return True
                else:
                    visited.add(neighbor)
                    queue.append(neighbor)

        return False
    
    graph = buildGraph(numCourses, prerequisites)
    return not hasCycle(graph)

# test
numCourses = 2
prerequisites = [[1,0]]
print(canFinish(numCourses, prerequisites))

numCourses = 2
prerequisites = [[1,0],[0,1]]
print(canFinish(numCourses, prerequisites))

numCourses = 4
prerequisites = [[1,0],[2,0],[3,1],[3,2]]
print(canFinish(numCourses, prerequisites))

True
False
True


## DFS Solution

Simply check if a node is visited or not. If it is visited, then there is a loop in the graph.

**follow up**: 

if we want to know the path of the loop, 
- we can use a `parent` dictionary to keep track of the parent of each node. Then we can use the parent dictionary to trace back the path of the loop.
- add a path parameter to the dfs function, and append the current node to the path. If there is a loop, then the path will contain the loop.


**notes**

think about when and how to use `visited` or `traced`.

- `visited`: the visited records all the nodes that have been visited, and explored. If during the recursion, the node marked as `visited` is visited again, there is no need to further do the calculation as it's been done before.
    - or we can understand it as: `visited` is for the global graph point of view to make sure the traversal is not performed twice for the same node. n
- `traced`: the traced record all the nodes that are currently on the recursion path, which might not be visited/finished. 
        - or we can understand it as: `traced` is the current recursion path, and can be used to see if current path has loop.

In [45]:
import collections
def canFinish(numCourses, prerequisites):

    def buildGraph(numCourses, prerequisites):
        # define graph as a dictionary with key as course and value as list of prerequisites
        graph = collections.defaultdict(list)
        for x, y in prerequisites:
            graph[x].append(y)
        return graph
    
    # store the visited nodes
    visited = {}
    traced = {}

    def hasCycle_dfs(graph, source):
        # if source is already in traced, then there is a cycle
        if source in traced:
            return True
            
        # if source is already visited, then there is no cycle
        if source in visited:
            return 

        # add source to visited
        visited[source] = True
        
        # add source to traced
        traced[source] = True 

        # explore all the neighbors of the source
        for neighbor in graph[source]:
            hasCycle_dfs(graph, neighbor)
        
        # remove source from traced
        traced.pop(source)

    graph = buildGraph(numCourses, prerequisites)
    print("graph is:", graph)
    for i in range(numCourses):
        
        hasCycle = hasCycle_dfs(graph, i)
        print(i, visited)
        if hasCycle:
            return not hasCycle
   
    return True
    
# test
numCourses = 2
prerequisites = [[1,0]]
print(canFinish(numCourses, prerequisites))

numCourses = 2
prerequisites = [[1,0],[0,1]]
print(canFinish(numCourses, prerequisites))

numCourses = 4
prerequisites = [[1,0],[2,0],[3,1],[3,2],[2,3],[1,3]]
print(canFinish(numCourses, prerequisites))

numCourses = 4
prerequisites = [[1,0],[2,0],[2,1],[3,1],[3,2]]
print(canFinish(numCourses, prerequisites))

graph is: defaultdict(<class 'list'>, {1: [0]})
0 {0: True}
1 {0: True, 1: True}
True
graph is: defaultdict(<class 'list'>, {1: [0], 0: [1]})
0 {0: True, 1: True}
1 {0: True, 1: True}
True
graph is: defaultdict(<class 'list'>, {1: [0, 3], 2: [0, 3], 3: [1, 2]})
0 {0: True}
1 {0: True, 1: True, 3: True, 2: True}
2 {0: True, 1: True, 3: True, 2: True}
3 {0: True, 1: True, 3: True, 2: True}
True
graph is: defaultdict(<class 'list'>, {1: [0], 2: [0, 1], 3: [1, 2]})
0 {0: True}
1 {0: True, 1: True}
2 {0: True, 1: True, 2: True}
3 {0: True, 1: True, 2: True, 3: True}
True


**Why the following code is correct??**

In [46]:
import collections
def canFinish(numCourses, prerequisites):

    def buildGraph(numCourses, prerequisites):
        # define graph as a dictionary with key as course and value as list of prerequisites
        graph = collections.defaultdict(list)
        for x, y in prerequisites:
            graph[x].append(y)
        return graph
    
    # store the visited nodes
    traced = {}

    def hasCycle_dfs(graph, source):
        if source in traced:
            return True

        # add source to traced
        traced[source] = True 

        # explore all the neighbors of the source
        for neighbor in graph[source]:
            hasCycle_dfs(graph, neighbor)
        
        # remove source from traced
        #traced.pop(source)

    graph = buildGraph(numCourses, prerequisites)
    print("graph is:", graph)
    for i in range(numCourses):
        
        hasCycle = hasCycle_dfs(graph, i)
        print(i, traced)
        if hasCycle:
            return not hasCycle
   
    return True
    
# test
numCourses = 2
prerequisites = [[1,0]]
print(canFinish(numCourses, prerequisites))

numCourses = 2
prerequisites = [[1,0],[0,1]]
print(canFinish(numCourses, prerequisites))

numCourses = 4
prerequisites = [[1,0],[2,0],[3,1],[3,2],[2,3],[1,3]]
print(canFinish(numCourses, prerequisites))

numCourses = 4
prerequisites = [[1,0],[2,0],[2,1],[3,1],[3,2]]
print(canFinish(numCourses, prerequisites))

graph is: defaultdict(<class 'list'>, {1: [0]})
0 {0: True}
1 {0: True, 1: True}
True
graph is: defaultdict(<class 'list'>, {1: [0], 0: [1]})
0 {0: True, 1: True}
1 {0: True, 1: True}
False
graph is: defaultdict(<class 'list'>, {1: [0, 3], 2: [0, 3], 3: [1, 2]})
0 {0: True}
1 {0: True, 1: True, 3: True, 2: True}
2 {0: True, 1: True, 3: True, 2: True}
False
graph is: defaultdict(<class 'list'>, {1: [0], 2: [0, 1], 3: [1, 2]})
0 {0: True}
1 {0: True, 1: True}
2 {0: True, 1: True, 2: True}
3 {0: True, 1: True, 2: True, 3: True}
True


If we also want to know the loop:

In [44]:
import collections
def canFinish(numCourses, prerequisites):

    def buildGraph(numCourses, prerequisites):
        # define graph as a dictionary with key as course and value as list of prerequisites
        graph = collections.defaultdict(list)
        for x, y in prerequisites:
            graph[x].append(y)
        return graph
    
    def hasCycle_dfs(graph, source, traced):
        if source in traced:
            # find cycle, then return the cycle
            loops.append(traced[:]+[source])

            return
        
        # add source to visited
        traced.append(source)

        # explore all the neighbors of the source
        for neighbor in graph[source]:
            hasCycle_dfs(graph, neighbor, traced)
        
        # remove from traced
        traced.pop()

    graph = buildGraph(numCourses, prerequisites)
    loops = []
    for i in range(numCourses):
        traced = []
        hasCycle_dfs(graph, i, traced)
        
    return loops
    
# test
numCourses = 2
prerequisites = [[1,0]]
print(canFinish(numCourses, prerequisites))

numCourses = 2
prerequisites = [[1,0],[0,1]]
print(canFinish(numCourses, prerequisites))

numCourses = 4
prerequisites = [[1,0],[2,0],[3,1],[3,2],[2,3],[1,3]]
print(canFinish(numCourses, prerequisites))

numCourses = 4
prerequisites = [[1,0],[2,0],[2,1],[3,1],[3,2]]
print(canFinish(numCourses, prerequisites))

[]
[[0, 1, 0], [1, 0, 1]]
[[1, 3, 1], [1, 3, 2, 3], [2, 3, 1, 3], [2, 3, 2], [3, 1, 3], [3, 2, 3]]
[]
